## Authorization to the DEDL services

In [1]:
from dedllogin import DEDL_auth
import requests
import json
import os
from getpass import getpass
from IPython.display import JSON

username = input("Please input your DESP username or email: ")
password = getpass("Please input your DESP password: ")

auth = DEDL_auth(username, password)

access_token = auth.get_token()
if access_token is not None:
    print("DEDL/DESP Access Token Obtained Successfully")
else:
    print("Failed to Obtain DEDL/DESP Access Token")

auth_headers = {"Authorization": f"Bearer {access_token}"}

Please input your DESP username or email:  jesper.dejby@ri.se
Please input your DESP password:  ········


DEDL/DESP Access Token Obtained Successfully


## Download data

We are aiming to download the Global Ocean Daily Gridded Surface Wind dataset described here: https://hda.data.destination-earth.eu/ui/dataset/EO.MO.DAT.WIND_GLO_PHY_L3_NRT_012_002

In [3]:
response_sws = requests.post("https://hda.data.destination-earth.eu/stac/search", headers=auth_headers, json={
    "collections": ["EO.MO.DAT.WIND_GLO_PHY_L3_NRT_012_002"],
    "datetime": "2024-05-31T00:00:00Z/2024-06-01T00:00:00Z",
    'bbox': [10,53,30,66]
})

In [5]:
from IPython.display import JSON
JSON(response_sws.json())

<IPython.core.display.JSON object>

In [6]:
response_sws.json()["features"][0]["assets"]["downloadLink"]["href"]

'https://hda.data.destination-earth.eu/stac/collections/EO.MO.DAT.WIND_GLO_PHY_L4_NRT_012_004/items/cmems_obs-wind_glo_phy_nrt_l4_0/download?provider=cop_marine'

In [20]:
product_sws = []
assets = []
for i in response_sws.json()["features"]:
    # Download NC
    #product_sws.append(i["assets"]["downloadLink"]["alternate"]["origin"]["href"])
    # Download using main link
    product_sws.append(i["assets"]["downloadLink"]["href"])
    assets.append(i["id"])

In [22]:
from tqdm import tqdm
import time

# Define a list of assets to download
download_url = product_sws[0]
filename = assets[0]
response = requests.get(download_url, headers=auth_headers)
total_size = int(response.headers.get("content-length", 0))

print(f"downloading {filename}")

with tqdm(total=total_size, unit="B", unit_scale=True) as progress_bar:
    with open(filename, 'wb') as f:
        for data in response.iter_content(1024):
            progress_bar.update(len(data))
            f.write(data)

downloading cmems_obs-wind_glo_phy_nrt_l4_0


73.3MB [00:00, 490MB/s]


In [23]:
product_sws

['https://hda.data.destination-earth.eu/stac/collections/EO.MO.DAT.WIND_GLO_PHY_L4_NRT_012_004/items/cmems_obs-wind_glo_phy_nrt_l4_0/download?provider=cop_marine',
 'https://hda.data.destination-earth.eu/stac/collections/EO.MO.DAT.WIND_GLO_PHY_L4_NRT_012_004/items/cmems_obs-wind_glo_phy_nrt_l4_0/download?provider=cop_marine',
 'https://hda.data.destination-earth.eu/stac/collections/EO.MO.DAT.WIND_GLO_PHY_L4_NRT_012_004/items/cmems_obs-wind_glo_phy_nrt_l4_0/download?provider=cop_marine',
 'https://hda.data.destination-earth.eu/stac/collections/EO.MO.DAT.WIND_GLO_PHY_L4_NRT_012_004/items/cmems_obs-wind_glo_phy_nrt_l4_0/download?provider=cop_marine',
 'https://hda.data.destination-earth.eu/stac/collections/EO.MO.DAT.WIND_GLO_PHY_L4_NRT_012_004/items/cmems_obs-wind_glo_phy_nrt_l4_0/download?provider=cop_marine',
 'https://hda.data.destination-earth.eu/stac/collections/EO.MO.DAT.WIND_GLO_PHY_L4_NRT_012_004/items/cmems_obs-wind_glo_phy_nrt_l4_0/download?provider=cop_marine',
 'https://hda.da

## Read data

We read the data and subset it too the baltic sea region.

In [24]:
import rioxarray
import matplotlib.pyplot as plt
import xarray as xr
import numpy as np
import netCDF4 as nc

In [29]:
ds = xr.open_dataset("cmems_obs-wind_glo_phy_nrt_l4_0")

In [30]:
ds

<xarray.Dataset>
Dimensions:                         (time: 1, lat: 1440, lon: 2880)
Coordinates:
  * time                            (time) datetime64[ns] 2022-09-01
  * lat                             (lat) float32 -89.94 -89.81 ... 89.81 89.94
  * lon                             (lon) float32 -179.9 -179.8 ... 179.8 179.9
Data variables: (12/27)
    eastward_wind                   (time, lat, lon) float32 ...
    eastward_wind_bias              (time, lat, lon) float32 ...
    eastward_wind_sdd               (time, lat, lon) float32 ...
    northward_wind                  (time, lat, lon) float32 ...
    northward_wind_bias             (time, lat, lon) float32 ...
    northward_wind_sdd              (time, lat, lon) float32 ...
    ...                              ...
    stress_curl                     (time, lat, lon) float64 ...
    stress_curl_bias                (time, lat, lon) float64 ...
    stress_curl_dv                  (time, lat, lon) float64 ...
    air_density                     (time, lat, lon) float32 ...
    number_of_observations          (time, lat, lon) float32 ...
    number_of_observations_divcurl  (time, lat, lon) float32 ...
Attributes: (12/26)
    title:                       Global Ocean - Wind and Stress - Hourly - Fr...
    summary:                    Global ocean 10-m stress-equivalent wind and ...
    keywords:                   ocean winds, wind speed, wind direction, wind...
    Conventions:                CF-1.6, ACDD-1.3
    project:                    Copernicus Marine Service Wind Thematic Assem...
    institution:                Royal Netherlands Meteorological Institute (K...
    ...                         ...
    time_coverage_start:        2022-09-01T00:00:00
    time_coverage_end:          2022-09-01T00:00:00
    references:                 Copernicus Marine Service Product User Manual...
    history:                    N/A
    date_created:               2022-09-02T10:31:44
    date_modified:              2022-09-02T10:31:44

In [34]:
lat_bnds, lon_bnds = [53, 66], [10, 30]
baltic_sea_subset=ds.sel(lat=slice(*lat_bnds), lon=slice(*lon_bnds))

In [35]:
baltic_sea_subset

<xarray.Dataset>
Dimensions:                         (time: 1, lat: 104, lon: 160)
Coordinates:
  * time                            (time) datetime64[ns] 2022-09-01
  * lat                             (lat) float32 53.06 53.19 ... 65.81 65.94
  * lon                             (lon) float32 10.06 10.19 ... 29.81 29.94
Data variables: (12/27)
    eastward_wind                   (time, lat, lon) float32 ...
    eastward_wind_bias              (time, lat, lon) float32 ...
    eastward_wind_sdd               (time, lat, lon) float32 ...
    northward_wind                  (time, lat, lon) float32 ...
    northward_wind_bias             (time, lat, lon) float32 ...
    northward_wind_sdd              (time, lat, lon) float32 ...
    ...                              ...
    stress_curl                     (time, lat, lon) float64 ...
    stress_curl_bias                (time, lat, lon) float64 ...
    stress_curl_dv                  (time, lat, lon) float64 ...
    air_density                     (time, lat, lon) float32 ...
    number_of_observations          (time, lat, lon) float32 ...
    number_of_observations_divcurl  (time, lat, lon) float32 ...
Attributes: (12/26)
    title:                       Global Ocean - Wind and Stress - Hourly - Fr...
    summary:                    Global ocean 10-m stress-equivalent wind and ...
    keywords:                   ocean winds, wind speed, wind direction, wind...
    Conventions:                CF-1.6, ACDD-1.3
    project:                    Copernicus Marine Service Wind Thematic Assem...
    institution:                Royal Netherlands Meteorological Institute (K...
    ...                         ...
    time_coverage_start:        2022-09-01T00:00:00
    time_coverage_end:          2022-09-01T00:00:00
    references:                 Copernicus Marine Service Product User Manual...
    history:                    N/A
    date_created:               2022-09-02T10:31:44
    date_modified:              2022-09-02T10:31:44

## Final remarks

There is a lot of data points here that we could consider in the model. In the end, we decided to limit our scope and not include a sea wind speed dataset.